<a href="https://colab.research.google.com/github/ssvadla/Demonstration/blob/main/SemiSupervised_Lightgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Semi supervised unlabelled code with Light gbm classifier without upsampling or downsampling.

In [25]:
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

train = pd.read_csv('/content/drive/My Drive/Research/train_data1.csv')
train.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,Sentence,Target
0,659,Appellant had stated to the officers that she ...,Invalid
1,3456,We shall discuss the facts more fully in conne...,Others
2,2043,"â€œPerjury is a false statement, either writte...",Invalid
3,3344,The offense is felony theft by false pretext; ...,Issue
4,3231,Numerous contentions urging the commission of ...,Issue


In [26]:
Threshold= 0.98

In [27]:
train['Target'].unique()

array(['Invalid', 'Others', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [28]:
train['Target']=train['Target'].replace(['Others'],'Invalid')
train['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [29]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)
X_tfidf = tfidf_vect.fit_transform(train['Sentence'])
print(X_tfidf.shape)

(3476, 6383)


In [31]:
test = pd.read_csv(r'/content/drive/My Drive/Research/test_data.csv')

test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))
t_p = tfidf_vect.transform(test['Sentence'])

In [32]:

from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import lightgbm as lgb


X, y = make_classification(random_state=1)

X_train, x_val, Y_train, y_val = train_test_split(X_tfidf,train['Target'],test_size=0.26,random_state=42)

classifier = lgb.LGBMClassifier()
classifier.fit(X_train, Y_train)
y_pred = classifier.predict(x_val)
#print(np.unique(y_pred))
#print(type(y_pred))

Accuracy_score = accuracy_score(y_pred,y_val)

print('Validation Accuracy_score: %f' % Accuracy_score)
test_pred = classifier.predict(t_p)
acc_test = accuracy_score(test_pred,test['Target'])
print('Accuracy test data', acc_test)
matrix = confusion_matrix(y_pred, y_val)
print(matrix)


Validation Accuracy_score: 0.642699
Accuracy test data 0.5777351247600768
[[ 43   6  27   8   8  18]
 [  6  21   4   2   0   0]
 [ 61  12 410  46  12  17]
 [ 11   6  35  66   4   8]
 [  2   1   6   3  25   5]
 [  3   0   3   7   2  16]]


In [33]:
train = train.rename(columns={'Sentence':'text'})
train.head()

,Unnamed: 0,text,Target
0,659,Appellant had stated to the officers that she ...,Invalid
1,3456,We shall discuss the facts more fully in conne...,Invalid
2,2043,"â€œPerjury is a false statement, either writte...",Invalid
3,3344,The offense is felony theft by false pretext; ...,Issue
4,3231,Numerous contentions urging the commission of ...,Issue


In [34]:

unlabel = pd.read_csv(r'/content/drive/My Drive/Research/Unlabeled_data.csv')
unlabel.head()


,Unnamed: 0,text,Complete
0,0,No motion for rehearing filed.,1
1,1,— Relator is under accusation of killing her h...,1
2,2,From an order of the District Judge of the 39t...,1
3,3,Relator and her husband had been married for a...,1
4,4,The evidence indicates that their family relat...,1


In [35]:
del unlabel['Complete']

In [36]:
del unlabel['Unnamed: 0']

In [37]:
unlabel.head()

,text
0,No motion for rehearing filed.
1,— Relator is under accusation of killing her h...
2,From an order of the District Judge of the 39t...
3,Relator and her husband had been married for a...
4,The evidence indicates that their family relat...


In [38]:
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
unlabel['text'] = unlabel['text'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x for x in x.split() if x not in words))


In [39]:
from textblob import TextBlob
from textblob import Word
nltk.download('wordnet')
nltk.download('punkt')
unlabel['text'] = unlabel['text'].apply(lambda x: TextBlob(x).words)
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [40]:
unlabel.head()

,text
0,motion rehearing filed
1,relator accusation killing husband virgil clark
2,order district judge 39th judicial district re...
3,relator husband married number year
4,evidence indicates family relation always plea...


In [41]:
len(unlabel)

537703

In [42]:
unlabel_1 = unlabel.loc[:100000]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = unlabel.loc[100001:200000]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = unlabel.loc[200001:300000]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = unlabel.loc[300001:400000]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = unlabel.loc[400001:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 100001
length of unlabel_2 100000
length of unlabel_3 100000
length of unlabel_4 100000
length of unlabel_5 137702


In [43]:
def index_reset(unlabel_2):
  unlabel_2.reset_index(inplace=True)
  del unlabel_2['index']
  print(unlabel_2.head())
  return unlabel_2

In [44]:
unlabel_1 = index_reset(unlabel_1)
unlabel_2 = index_reset(unlabel_2)
unlabel_3 = index_reset(unlabel_3)
unlabel_4 = index_reset(unlabel_4)
unlabel_5 = index_reset(unlabel_5)

                                                text
0                             motion rehearing filed
1    relator accusation killing husband virgil clark
2  order district judge 39th judicial district re...
3                relator husband married number year
4  evidence indicates family relation always plea...
                                                text
0  responding conclusion regarding evidence annou...
1  conviction transporting intoxicating liquor pu...
2                     bill exception case filed late
3  motion new trial overruled october 22 1927 ord...
4  thereafter december 10th extension order made ...
                                                text
0                       another man carrying clothes
1  officer testified appellant several item cloth...
2  investigation followed report breaking entry a...
3                                  appellant present
4  warrant arrest issued officer went hotel waite...
                                              

In [45]:
classifier.classes_

array(['Analysis', 'Conclusion', 'Facts', 'Invalid', 'Issue',
       'Rule/Law/Holding'], dtype=object)

In [46]:

Accuracy_Valid_list = []
Accuracy_Test_list = []


def training_model(classifier_1, train_1, unlabel_2 ):

  x_un2 = tfidf_vect.transform(unlabel_2['text'])

  pred_unlabel_2 = classifier_1.predict_proba(x_un2)
  pred_unlabel_2

  print(len(pred_unlabel_2))
  print(np.unique(pred_unlabel_2))

  
  pos=[]
  large=[]
  ind = []
  i=0
  for j in pred_unlabel_2:
    if max(j)> Threshold:
      ind.append(np.argmax(j))
      large.append(max(j))
      pos.append(i)
    i+=1


  print(ind)
  print(large)
  print(pos)
  print(type(pos))
  print(len(ind))
  print(len(large))
  print(len(pos))

  unlabel_2 = unlabel_2.loc[pos,:]
  print(len(unlabel_2))
  unlabel_2.head()


  class_x_un2 = tfidf_vect.transform(unlabel_2['text'])

  #class_x_un1.shape

  class_pred_unlabel_2 = classifier_1.predict(class_x_un2)
  class_pred_unlabel_2

  print(len(class_pred_unlabel_2))

  unlabel_2['Target']=class_pred_unlabel_2
  unlabel_2.head()

  #train_1.head()

  frame_2 = [train_1,unlabel_2]
  train_2 = pd.concat(frame_2)
  print(len(train_1))
  print(len(unlabel_2))
  print(len(train_2))

  train_2

  
  x_train_2 = tfidf_vect.transform(train_2['text'])
  x_train_2.shape
  X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train_2['Target'],test_size=0.25,random_state=2)
  classifier_2 = lgb.LGBMClassifier()
  classifier_2.fit(X_train, Y_train)
  y_pred = classifier_2.predict(x_val)
  print(np.unique(y_pred))
  print(type(y_pred))


  Accuracy_score = accuracy_score(y_pred,y_val)
  Accuracy_Valid_list.append(Accuracy_score)

  print('Validation ------- Accuracy_score: %f' % Accuracy_score)

  test_pred = classifier_2.predict(t_p)
  acc_test = accuracy_score(test_pred,test['Target'])
  Accuracy_Test_list.append(acc_test)

  print('Test ------Accuracy score ', acc_test)
  matrix = confusion_matrix(y_pred, y_val)
  print(matrix)
  matrix = confusion_matrix(test_pred, test['Target'])
  print(matrix)
  print("---------validation--------------")
  print(classification_report(y_val,y_pred))
  print("----------test---------------------")
  print(classification_report(test['Target'],test_pred))
  return classifier_2, train_2


In [47]:
classifier_1,train_1 = training_model(classifier, train, unlabel_1 )
classifier_2,train_2 = training_model(classifier_1, train_1, unlabel_2 )
classifier_3,train_3 = training_model(classifier_2, train_2, unlabel_3 )
classifier_4,train_4 = training_model(classifier_3, train_3, unlabel_4 )
classifier_5,train_5 = training_model(classifier_4, train_4, unlabel_5 )

100001
[3.98327271e-06 5.18122818e-06 5.37703679e-06 ... 9.99654837e-01
 9.99690707e-01 9.99733617e-01]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

In [48]:
print(Accuracy_Valid_list)
print(Accuracy_Test_list)

[0.8158673771462404, 0.9195201199700075, 0.9626692692448456, 0.9779858480451719, 0.9867310518580726]
[0.6026871401151631, 0.6007677543186181, 0.6218809980806143, 0.5988483685220729, 0.6007677543186181]
